In [1]:
!pip install transformers
!pip install sentencepiece
!pip install torchmetrics
!pip install pytorch_lightning
!pip install wandb
!wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 34.0 MB/s 
     |████████████████████████████████| 182 kB 52.7 MB/s 
     |████████████████████████████████| 7.6 MB 55.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 30.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 19.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 10.3 MB/s 
     |████████████████████████████████| 87 kB 4.0 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=2864fd57f02896cae19c79da4dd9201d69163b52d28d257d22f5bd6eca588286
  Stored in directory: /root/.cache/pip/wheels/8a

In [2]:
import json
from functools import partial

#from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
from torch.optim import AdamW
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

from pytorch_lightning.loggers import WandbLogger

from sklearn.model_selection import train_test_split

In [3]:
N_GPUS = torch.cuda.device_count()
N_GPUS

1

In [4]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
encoder = AutoModel.from_pretrained('distilbert-base-multilingual-cased')
#tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
#encoder = XLMRobertaModel.from_pretrained('xlm-roberta-base')
encoder

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(

In [5]:
EXP_NAME = 'mlns-distilbert-regressor'
TRAIN_BATCH_SIZE = 2 
DEV_BATCH_SIZE = 64
ACCUMULATE_GRAD = 8

#Model

In [6]:
class SimilarityRegressor(nn.Module):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(SimilarityRegressor, self).__init__()

        self.encoder = encoder
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        
        self.linear1 = nn.Linear(self.embed_size, self.hidden_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        self.dropout1 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(2*self.hidden_size, self.hidden_size//2)
        self.dropout2 = nn.Dropout(p=0.2)
        self.activation2 = nn.LeakyReLU(negative_slope=0.1)
        self.linear3 = nn.Linear(self.hidden_size//2, 1)
        #self.activation3 = nn.Sigmoid()

    def common_compute(self, x):
        x = self.encoder(**x)[0][:, 0]
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        return x
    
    def forward(self, x1, x2):
        x1 = self.common_compute(x1)
        x2 = self.common_compute(x2)
        x = torch.cat([torch.abs(x1 - x2), (x1 + x2)], dim=-1)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        #x = 3*self.activation3(x) + 1

        return x

In [7]:
class LitSimilarityRegressor(pl.LightningModule):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(LitSimilarityRegressor, self).__init__()
        self.model = SimilarityRegressor(encoder, embed_size=embed_size, hidden_size=hidden_size)

        self.train_loss = torchmetrics.MeanMetric(compute_on_step=True)
        self.dev_loss = torchmetrics.MeanMetric(compute_on_step=False)
        self.test_loss = torchmetrics.MeanMetric(compute_on_step=False)

        self.train_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=True)
        self.dev_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)
        self.test_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)

        self.train_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=True)
        self.dev_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)
        self.test_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)

    def forward(self, x1, x2):
        return self.model(x1, x2)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5, betas=(0.9, 0.99), eps=1e-8, weight_decay=0.01)

    def training_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def validation_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}
    
    def test_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def predict_step(self, batch, batch_idx):
        x1, x2, _ = batch
        output = self(x1, x2)

        return {'preds': output}

    def training_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.log('train/step/loss', self.train_loss(loss))
        self.log('train/step/mape', self.train_mape(preds, target))
        self.log('train/step/pcc', self.train_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,))))

    def validation_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.dev_loss(loss)
        self.dev_mape(preds, target)
        self.dev_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def test_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.test_loss(loss)
        self.test_mape(preds, target)
        self.test_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def training_epoch_end(self, outs):
        self.log('train/epoch/loss', self.train_loss)
        self.log('train/epoch/mape', self.train_mape)
        self.log('train/epoch/pcc', self.train_pcc)

    def validation_epoch_end(self, outs):
        self.log('dev/loss', self.dev_loss)
        self.log('dev/mape', self.dev_mape)
        self.log('dev/pcc', self.dev_pcc)
    
    def test_epoch_end(self, outs):
        self.log('dev/loss', self.test_loss)
        self.log('dev/mape', self.test_mape)
        self.log('dev/pcc', self.test_pcc)

In [8]:
model = LitSimilarityRegressor(encoder, embed_size=768, hidden_size=256)
model

LitSimilarityRegressor(
  (model): SimilarityRegressor(
    (encoder): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
     

#Data

In [9]:
#Link google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
root_path = 'gdrive/My Drive/IIIT/Sem1/IRE/Major_project/'
train_data = json.load(open(root_path+'train_data.json', 'r'))
test_data = json.load(open(root_path+'test_data.json', 'r'))

In [11]:
train_dataset, dev_dataset = train_test_split(train_data, test_size=0.20, random_state=42, shuffle=True)
test_dataset = test_data

In [12]:
test_data[0]

{'pair_id': '1486865227_1542306554',
 'title_1': '3 ابتدائي.. توزيع أوراق امتحان نهاية العام بدلاً من نصف العام في مدارس بالمنصورة',
 'title_2': 'حقيقة منشور شرق المنصورة التعليمية المثير للجدل بمنع الرحلات والزيارات',
 'text_1': 'فوجئ تلاميذ الصف الثالث الابتدائى في 5 مدارس بإدارة شرق المنصورة عقب توزيع\n\nأوراق\n\nامتحان الصف الثالث الابتدائى الفترة الأولى مادة التربية الدينية بأن الامتحان من منهج الترم الثانى.\n\nأدى ذلك الى حالة من الارتباك بين الطلاب وعلى الفور تم توزيع البديل على الطلاب وإعطاء وقت إضافى للطلبة وتم تحويل الواقعة للشئون القانونية لاتخاذ اللازم.\n\n\n\n\n\nوقرّر المهندس على عبد الرؤوف وكيل وزارة التربية والتعليم بالدقهلية إحالة\n\nالمسئولين\n\nعن واقعة توزيع امتحان نهاية العام لمادة التربية الدينية للصف الثالث الابتدائى\n\nبإدارة\n\nشرق المنصورة بدلاً من الفصل الدراسى الاول للشئون\n\nالقانونية\n\nللتحقيق واتخاذ\n\nالإجراءات\n\nالقانونية\n\n.',
 'text_2': 'أصدرت إدارة شرق المنصورة التعليمية التابعة لمديرية التربية والتعليم بالدقهلية، اليوم الأحد، منشورا رقم 100046، ت

In [13]:
def collate_fn(batch, tokenizer):
    texts_1, texts_2, scores = list(), list(), list()
    for sample in batch:

        text1 = str(sample['text_1']).lower().strip()
        text2 = str(sample['text_2']).lower().strip()
        
        score = torch.tensor([sample['score']])
        texts_1.append(text1)
        texts_2.append(text2)
        scores.append(score)

    texts_1 = tokenizer(texts_1, truncation=True, padding=True, return_tensors='pt')
    texts_2 = tokenizer(texts_2, truncation=True, padding=True, return_tensors='pt')
    scores = torch.cat(scores, dim=0).unsqueeze(1)

    return texts_1, texts_2, scores

In [14]:
collate_partial = partial(collate_fn, tokenizer=tokenizer)
dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_partial)
next(iter(dataloader))

({'input_ids': tensor([[  101, 11859, 94096,  ...,   117, 10339,   102],
         [  101, 16065, 14234,  ..., 10992, 71472,   102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])},
 {'input_ids': tensor([[  101,   100,   187,  ..., 28780, 34965,   102],
         [  101, 19129,   120,  ..., 14591, 86957,   102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])},
 tensor([[1.0000],
         [3.3333]]))

In [15]:
class MLNSDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, dev_dataset, test_dataset, train_batch_size, dev_batch_size, collate_fn, tokenizer):
        super(MLNSDataModule, self).__init__()
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset
        self.train_batch_size = train_batch_size
        self.dev_batch_size = dev_batch_size
        self.collate_fn = collate_fn
        self.tokenizer = tokenizer

    def train_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.train_batch_size, collate_fn=collate_partial)

    def val_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.dev_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def test_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def predict_dataloader(self):
        #return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)
        return self.test_dataloader()

In [16]:
data_module = MLNSDataModule(train_dataset, dev_dataset, test_dataset, TRAIN_BATCH_SIZE, DEV_BATCH_SIZE, collate_fn=collate_fn, tokenizer=tokenizer)
data_module

#Training

In [17]:
logger = WandbLogger(save_dir=EXP_NAME, project=EXP_NAME, log_model=False)
logger

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING Path mlns-distilbert-regressor/wandb/ wasn't writable, using system temp directory.


In [18]:
checkpoint_callback = ModelCheckpoint(
    dirpath=root_path+'model',
    filename='{epoch}-{step}',
    monitor='dev/pcc',
    mode='max',
    save_top_k=1,
    verbose=True,
    save_last=False,
    save_weights_only=False,
    every_n_epochs=1
)

In [19]:
trainer = pl.Trainer(max_epochs=10,
                     accumulate_grad_batches=ACCUMULATE_GRAD,
                     accelerator='gpu',
                     check_val_every_n_epoch=1, val_check_interval=0.25,
                     enable_progress_bar=True,
                     gradient_clip_val=0.25, track_grad_norm=2,
                     enable_checkpointing=True,
                     callbacks=[checkpoint_callback],
                     logger=logger,
                     enable_model_summary=True)
                    

trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(model, datamodule=data_module)
#trainer.fit(model, datamodule=data_module, ckpt_path=root_path+'model/epoch=3-step=571.ckpt')

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                        | Params
-----------------------------------------------------------
0 | model      | SimilarityRegressor         | 134 M 
1 | train_loss | MeanMetric                  | 0     
2 | dev_loss   | MeanMetric                  | 0     
3 | test_loss  | MeanMetric                  | 0     
4 | train_mape | MeanAbsolutePercentageError | 0     
5 | dev_mape   | MeanAbsolutePercentageError | 0     
6 | test_mape  | MeanAbsolutePercentageError | 0     
7 | train_pcc  | PearsonCorrCoef             | 0     
8 | dev_pcc    | PearsonCorrCoef             | 0     
9 | test_pcc   | PearsonCorrCoef             | 0     
-----------------------------------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
539.987   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 35: 'dev/pcc' reached -0.08247 (best -0.08247), saving model to '/content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=35.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 71: 'dev/pcc' reached -0.06432 (best -0.06432), saving model to '/content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=71.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 106: 'dev/pcc' reached 0.18922 (best 0.18922), saving model to '/content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=106.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 142: 'dev/pcc' reached 0.21954 (best 0.21954), saving model to '/content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=142.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [24]:
test_pred = trainer.predict(datamodule=data_module)

all_outputs = list()
for batch_outputs in test_pred:
    all_outputs.append(batch_outputs['preds'])
all_outputs = torch.cat(all_outputs, dim=0)

all_outputs.shape

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=142.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=142.ckpt


Predicting: 1143it [00:00, ?it/s]

torch.Size([4316, 1])

In [25]:
test_result = trainer.test(datamodule=data_module)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=142.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/gdrive/My Drive/IIIT/Sem1/IRE/Major_project/model/epoch=0-step=142.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        dev/loss            1.3572213649749756
        dev/mape            0.5982285141944885
         dev/pcc            0.1072964295744896
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [26]:
#test_result = trainer.test(model, datamodule=data_module, ckpt_path=root_path+'model/epoch=8-step=1179.ckpt', verbose=True)